In [ ]:
from common.amigos.dataset import AMIGOSDataset

ds = AMIGOSDataset("../../resources/AMIGOS/sampled/AMIGOS_sampled.csv")
ds.__getitem__(1)

In [ ]:
import numpy as np

a = ds.__getitem__(1)
for i in a:
    if i is None:
        print("None")
    else:
        print(i.shape if isinstance(i, np.ndarray) else len(i))

In [ ]:
len(a[0])

In [ ]:
np.array(a[0]).shape

In [ ]:
from sympy.printing.pytorch import torch
from torchviz import make_dot
from models.FEEG.model import EEGAVI

model = EEGAVI()
x = torch.randn(1, 8)
y = model(x)

make_dot(y.mean(), params=dict(model.named_parameters()), show_attrs=True, show_saved=True)